<a href="https://colab.research.google.com/github/NimraAslamkhan/segmentation-problem-classify-each-deep-learning-/blob/main/segmentation_problem_classify_each_deep_learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1**:
 **Set Up the** **Dataset**

The dataset contains the following directories and files

**class_dict_seg.csv:**

This likely contains information about the classes in the segmentation task.

**pixel_based_mask:**

Contains pixel-based masks.

**test_image:**

Contains test images.

**test_mask:**

Contains test masks.

**train_image:**

Contains training images.

**train_mask:**

Contains training masks.

proceed by setting up a tensorflow dataset and data loader to use this data for training and testing. Then will integrate the partial cross entropy loss function and train a U-Net model on this dataset.

In [2]:
from google.colab import files

uploaded = files.upload()




Saving archive (9).zip to archive (9) (1).zip


**Step 2: Preprocess  Data**


In [3]:
import numpy as np
import pandas as pd


In [6]:
# Randomly sample the DataFrame
sampled_df = df.sample(frac=sample_fraction, random_state=42)


In [7]:
print(f"Original data size: {len(df)}")
print(f"Sampled data size: {len(sampled_df)}")
print(sampled_df.head())


Original data size: 1000
Sampled data size: 100
            x         y  pixel_based_mask
521  0.184823  0.231353                 0
737  0.668804  0.850834                 1
740  0.846616  0.941640                 0
660  0.407528  0.489112                 1
411  0.701646  0.857795                 0


**Step 3:**

**Task 1:  Implement partial Cross entropy loss bY using tensorflow**

create a custom loss function that combines binary cross-entropy with a Dice coefficient loss, which is commonly used for segmentation tasks to handle class imbalance.

In [8]:
#Partial Cross-Entropy Loss
import tensorflow as tf
from tensorflow.keras import backend as K

def partial_cross_entropy(y_true, y_pred, class_weights, epsilon=1e-7):
    # Compute cross-entropy for each class
    cross_entropy = - y_true * K.log(y_pred + epsilon) - (1 - y_true) * K.log(1 - y_pred + epsilon)

    # Apply class weights
    weighted_cross_entropy = class_weights * cross_entropy

    # Average the loss across all classes and pixels
    return K.mean(weighted_cross_entropy)

**Task 2 : Build Your Segmentation Model**

Use a standard segmentation model such as U-Net or a custom architecture.

In [9]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

def unet_model(input_size=(256, 256, 3), num_classes=1):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    # Output layer
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(c5)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Create the model
model = unet_model(input_size=(256, 256, 3), num_classes=1)

# Compile the model with custom loss function
class_weights = [1.0]  # Example: Adjust weights based on your classes
model.compile(optimizer='adam', loss=lambda y_true, y_pred: partial_cross_entropy(y_true, y_pred, class_weights))

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 256, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)         0         ['conv2d_1[0][0]']            
 D)                                                                                           

In [17]:

# Save the trained model
model.save('final_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**I proceed by setting up a PyTorch , dataset and data loader to use this data for training and testing. Then, we will integrate the partial cross entropy loss function and train a U-Net model on this dataset.**

In [36]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [39]:
# Create dataset and dataloader
train_dataset = RemoteSensingDataset(
    image_dir=os.path.join("/content/extracted_images", 'train_image'),
    mask_dir=os.path.join("/content/extracted_images", 'train_mask'),
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Example of accessing the dataset
example_image, example_mask = next(iter(train_loader))
example_image.shape, example_mask.shape

(torch.Size([4, 3, 420, 420]), torch.Size([4, 1, 420, 420]))

**Step 2: Implement Partial Cross Entropy Loss**

In [40]:
import torch.nn.functional as F

class PartialCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(PartialCrossEntropyLoss, self).__init__()

    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=1)
        loss = -torch.sum(target * log_prob) / input.size(0)
        return loss




**Step 3: Define the U-Net Model**

already defined the U-Net model earlier by using tensorflow  . Here is a recap with  **pytorch**

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.center = self.conv_block(512, 1024)
        self.dec4 = self.conv_block(1024 + 512, 512)
        self.dec3 = self.conv_block(512 + 256, 256)
        self.dec2 = self.conv_block(256 + 128, 128)
        self.dec1 = self.conv_block(128 + 64, 64)
        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def crop(self, enc, dec):
        enc_size = enc.size()[2:]
        dec_size = dec.size()[2:]
        delta_h = enc_size[0] - dec_size[0]
        delta_w = enc_size[1] - dec_size[1]
        dec = F.pad(dec, (delta_w // 2, delta_w - delta_w // 2, delta_h // 2, delta_h - delta_h // 2))
        return dec

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, 2))
        enc3 = self.enc3(F.max_pool2d(enc2, 2))
        enc4 = self.enc4(F.max_pool2d(enc3, 2))
        center = self.center(F.max_pool2d(enc4, 2))

        dec4 = self.crop(enc4, F.interpolate(center, scale_factor=2, mode='bilinear', align_corners=True))
        dec4 = self.dec4(torch.cat([dec4, enc4], dim=1))

        dec3 = self.crop(enc3, F.interpolate(dec4, scale_factor=2, mode='bilinear', align_corners=True))
        dec3 = self.dec3(torch.cat([dec3, enc3], dim=1))

        dec2 = self.crop(enc2, F.interpolate(dec3, scale_factor=2, mode='bilinear', align_corners=True))
        dec2 = self.dec2(torch.cat([dec2, enc2], dim=1))

        dec1 = self.crop(enc1, F.interpolate(dec2, scale_factor=2, mode='bilinear', align_corners=True))
        dec1 = self.dec1(torch.cat([dec1, enc1], dim=1))

        final = self.final(dec1)
        return final

# Verify the model with a dummy input
model = UNet()
dummy_input = torch.randn(4, 3, 256, 256)
output = model(dummy_input)
output.shape



torch.Size([4, 1, 256, 256])

**Step 4: Train the Network**

In [46]:
import torch.optim as optim

# Instantiate model, loss, and optimizer
model = UNet()
criterion = PartialCrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/10, Loss: 0.0
Epoch 2/10, Loss: 0.0
Epoch 3/10, Loss: 0.0
Epoch 4/10, Loss: 0.0
Epoch 5/10, Loss: 0.0
Epoch 6/10, Loss: 0.0
Epoch 7/10, Loss: 0.0
Epoch 8/10, Loss: 0.0
Epoch 9/10, Loss: 0.0
Epoch 10/10, Loss: 0.0


**Step 5: Design and Run Experiments**

Experiment Setup

Learning Rates: Try 0.001, 0.0001, and 0.01.

Batch Sizes: Try 16, 32, and 64.



In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import matplotlib.pyplot as plt



In [17]:
# Define the neural network architecture
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, image_channels)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    return model

In [19]:
# Initialize variables for recording results
learning_rates = [0.001, 0.0001, 0.01]
batch_sizes = [16, 32, 64]
epochs = 10

results = {}



In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets  # Assuming you use torchvision for datasets

# Define your segmentation model (replace with your actual model architecture)
class SegmentationModel(nn.Module):
    def __init__(self):
        super(SegmentationModel, self).__init__()
        # Define your layers here, for example:
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 2, kernel_size=1)

    def forward(self, x):
        # Define the forward pass
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.conv3(x)
        return x

In [29]:
# Function to train the model
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [30]:
# Function to evaluate the model
def evaluate_model(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(val_loader)

In [31]:
# Main function to run the experiment
def main():
    # Define hyperparameters and configurations
    learning_rates = [0.001, 0.0001, 0.01]
    batch_sizes = [16, 32, 64]
    num_epochs = 10
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
# Iterate over different learning rates and batch sizes
for lr in learning_rates:
        for batch_size in batch_sizes:
            print(f"Training with learning rate: {lr}, batch size: {batch_size}")



Training with learning rate: 0.001, batch size: 16
Training with learning rate: 0.001, batch size: 32
Training with learning rate: 0.001, batch size: 64
Training with learning rate: 0.01, batch size: 16
Training with learning rate: 0.01, batch size: 32
Training with learning rate: 0.01, batch size: 64
Training with learning rate: 0.1, batch size: 16
Training with learning rate: 0.1, batch size: 32
Training with learning rate: 0.1, batch size: 64


In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Define your image classification model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to train the model
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Function to evaluate the model
def evaluate_model(model, val_loader, criterion, device):
    model.eval()
    total_correct = 0
    total_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    accuracy = total_correct / len(val_loader.dataset)
    return total_loss / len(val_loader), accuracy

# Main function to run the experiment
def main():
    # Define hyperparameters and configurations
    learning_rates = [0.001, 0.0001, 0.01]
    batch_sizes = [16, 32, 64]
    num_epochs = 10
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Define transforms and dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    val_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Iterate over different learning rates and batch sizes
    for lr in learning_rates:
        for batch_size in batch_sizes:
            print(f"Training with learning rate: {lr}, batch size: {batch_size}")

            # Initialize the model, optimizer, and criterion
            model = SimpleCNN().to(device)
            optimizer = optim.Adam(model.parameters(), lr=lr)
            criterion = nn.CrossEntropyLoss()

            # Training loop
            for epoch in range(num_epochs):
                train_loss = train_model(model, train_loader, optimizer, criterion, device)
                print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}")

            # Evaluation on validation set
            val_loss, val_accuracy = evaluate_model(model, val_loader, criterion, device)
            print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

if __name__ == "__main__":
    main()


100%|██████████| 170498071/170498071 [00:03<00:00, 48828150.94it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Training with learning rate: 0.001, batch size: 16
Epoch [1/10], Train Loss: 1.2856
Epoch [2/10], Train Loss: 0.9121
Epoch [3/10], Train Loss: 0.7604
Epoch [4/10], Train Loss: 0.6470
Epoch [5/10], Train Loss: 0.5487
Epoch [6/10], Train Loss: 0.4590
Epoch [7/10], Train Loss: 0.3857
Epoch [8/10], Train Loss: 0.3118
Epoch [9/10], Train Loss: 0.2522
Epoch [10/10], Train Loss: 0.2106
Validation Loss: 1.3445, Validation Accuracy: 0.7027
Training with learning rate: 0.001, batch size: 32
Epoch [1/10], Train Loss: 1.2580
Epoch [2/10], Train Loss: 0.8959
Epoch [3/10], Train Loss: 0.7363
Epoch [4/10], Train Loss: 0.6149
Epoch [5/10], Train Loss: 0.5083
Epoch [6/10], Train Loss: 0.4147
Epoch [7/10], Train Loss: 0.3314
Epoch [8/10], Train Loss: 0.2609
Epoch [9/10], Train Loss: 0.2077
Epoch [10/10], Train Loss: 0.1686
Validation Loss: 1.3422, Validation Accuracy: 0.7103
Training with learning rate: 0.001, batch